In [1]:
import json

In [3]:
f=json.load(open("/Users/ryzenx/Codes/Python/cc3dparser/temp_data/User Library-3DS Cart.artec_prt.json"))

In [12]:
extrude_entity=f['entities'][-1]

In [15]:
extrude_entity['feature']['extrude']['references']

['983A000001000000FFFEFF00000000007B010000']

In [16]:
id_="983A000001000000FFFEFF00000000007B010000"

In [17]:
def get_sketch_id(all_stat,sketch_id):
        for tm in all_stat["timeline"]:
            if tm['uuid']==sketch_id:
                return tm['index']
        return None

In [21]:
f['entities'][29]['sketch']['profiles']

[{'uuid': '983A000022000000FFFEFF00000000007B01000006000000FFFF01000C0073674C696E6548616E646C650000FFFFFFFF0000000001800200FFFFFFFF0000000001800300FFFFFFFF0000000001800400FFFFFFFF0000000001800600FFFFFFFF0000000001800700FFFFFFFF00000000',
  'curves': ['983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650000FFFFFFFF00000000',
   '983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650200FFFFFFFF00000000',
   '983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650300FFFFFFFF00000000',
   '983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650400FFFFFFFF00000000',
   '983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650600FFFFFFFF00000000',
   '983A000006000000FFFEFF00000000007B010000FFFF01000C0073674C696E6548616E646C650700FFFFFFFF00000000'],
  'isOuter': False,
  'isClosed': True}]